In [ ]:
from fastai.vision.all import *
import torch

In [ ]:
path = untar_data(URLs.PETS)/'images'
def is_cat(x):
    return x[0].isupper()
dls = ImageDataLoaders.from_name_func(path, get_image_files(path),valid_pct = 0.2, seed=21,
                                      label_func = is_cat, item_tfms = Resize(224))
learn = cnn_learner(dls, resnet34, metrics = error_rate)
learn.fine_tune(1)

/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:284: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627

  0%|          | 0.00/83.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.151478,0.019330,0.006089,32:29


epoch,train_loss,valid_loss,error_rate,time


In [ ]:
img = PILImage.create(image_cat())
x, = first(dls.test_dl([img]))

NameError: ignored

In [ ]:
class Hook():
    def hook_func(self, m, i, o):
        self.stored = o.detch().clone()

In [ ]:
hook_output = Hook()
hook = learn.model[0].register_forward_hook(hook_output.hook_func)

In [ ]:
with torch.no_grad():
    output = learn.model.eval()(x)

In [ ]:
act = hook_output.stored[0]

In [ ]:
F.softmax(output, dim = -1)

In [ ]:
dls.vocab

In [ ]:
act.shape

In [ ]:
cam_map = torch.einsum('ck, kij->cij', learn.model[1][-1].weight, act)
cam_map.shape

In [ ]:
x_dec = TensorImage(dls.train.decode((x,)[0][0]))
_,ax = plt.subplots()
x_dec.show(ctx = ax)
ax.image(cam_map[1].detach().cpu(), alpha = .6, extent = (0, 224,224,0), interpolation = 'bilinear', cmap='magma')